# Julia Dash Binder

A binder example demonstrating how to make a Julia notebook in appmode

-------

<span class="js-load-message cs-loading">This should only take a minute.</span>

In [ ]:
cur_script = """
    <style class="js-load-remove-tag">
        .cs-loading:after {
            overflow: hidden;
            display: inline-block;
            vertical-align: bottom;
            -webkit-animation: cs-ellipsis steps(3,end) 3250ms infinite;      
            animation: cs-ellipsis steps(3,end) 3250ms infinite;
            content: "\\2026";
            width: 0px;
        }
        @keyframes cs-ellipsis { to { width: 0.9em; } }
        @-webkit-keyframes cs-ellipsis { to { width: 0.9em; } }
    </style>

    <script class="js-load-remove-tag">
      \$(".js-load-remove-tag").parent(".output_subarea").css("padding", "0");
    </script>
"""

display(HTML(cur_script))

In [ ]:
using HTTP
using JSON

In [ ]:
get_data(cur_report::AbstractString, cur_api="api.juliatetris.com") = 
    JSON.parse(String(
        HTTP.get("http://$(cur_api)/reports/$(cur_report)").body
    ))

cur_score_data = get_data("score")
cur_round_data = get_data("round")

@assert cur_score_data["days"] == cur_round_data["days"]

cur_days = cur_score_data["days"]
num_days = length(cur_days)

return

In [ ]:
using WebIO
using JSExpr

In [ ]:
cur_scope = Scope(imports=[
    "https://fonts.googleapis.com/css?family=Roboto",
    "https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.7.2/Chart.bundle.min.js"
])

cur_set_types = ["scores", "counts"]

cur_scale_types = ["lin", "log"]

cur_set_observer = Observable(cur_scope, "data-set", first(cur_set_types))

cur_scale_observer = Observable(cur_scope, "data-scale", first(cur_scale_types))

return

In [ ]:
function make_buttons(cur_observer::Observable, cur_types::Vector)
    cur_buttons = []
    
    for (cur_index, cur_type) in enumerate(cur_types)
        cur_class = "btn btn-default"
        ( cur_index == 1 ) && ( cur_class *= " active" )
        
        push!(cur_buttons,
            Node(
                :div, cur_type, 
                attributes=Dict(:class => cur_class, :style => "border-radius: 0;"),
                events=Dict("click" => @js (cur_event) -> $cur_observer[] = cur_event.target.innerText)
            )
        )
    end
    
    cur_buttons
end

cur_left_buttons = make_buttons(cur_set_observer, cur_set_types)
cur_right_buttons = make_buttons(cur_scale_observer, cur_scale_types)

return

In [ ]:
function toggle_button(cur_action::AbstractString, cur_selector::AbstractString)
    cur_js = """
        \$("#js-julia-dash $(cur_selector) .btn").removeClass("active");
        \$("#js-julia-dash $(cur_selector) .btn:contains('$(cur_action)')").addClass("active");    
    """
    
    evaljs(cur_scope, JSString(cur_js))
end

return

In [ ]:
function get_cur_data_sets()
    cur_data_sets = []
    
    if cur_set_observer.val == "scores"
        cur_headers = ["Max Score", "Leaderboard Average"]
        push!(cur_data_sets, cur_score_data["max"])
        push!(cur_data_sets, cur_score_data["mean"])
    elseif cur_set_observer.val =="counts"
        cur_headers = ["Round Counts", "Saved Scores"]
        push!(cur_data_sets, cur_round_data["count"])
        push!(cur_data_sets, cur_score_data["count"])
    else
        error("Invalid data set selection.")
    end
    
    return cur_headers, cur_data_sets
end

return

In [ ]:
function get_cur_scale()
    if cur_scale_observer.val == "lin"
        cur_scale = "linear"
    elseif cur_scale_observer.val == "log"
        cur_scale = "logarithmic"
    else
        error("Invalid data scale selection.")
    end    
    
    cur_scale
end

return

In [ ]:
function run_javascript(cur_headers, cur_data_sets, cur_scale)
    cur_js = ""
    
    for (cur_index, cur_data) in enumerate(cur_data_sets)
        cur_js *= """
            var curChart = \$("#js-tetris-chart-$(cur_index)");
        
            var myLineChart = new Chart(curChart, {
                type: 'line',
                data: {
                    labels: $(JSON.json(cur_days)),
                    datasets: [{
                        data: $(JSON.json(cur_data)),
                        backgroundColor: [[
                            'rgba(255, 99, 132, 0.2)',
                            'rgba(54, 162, 235, 0.2)'
                        ][$(cur_index-1)]],
                        borderColor: [[
                            'rgba(255,99,132,1)',
                            'rgba(54, 162, 235, 1)'
                        ][$(cur_index-1)]]
                    }]
                },
                options: {
                    scales: {
                        xAxes: [{
                            type: 'time',
                            time: { unit: 'day' }
                        }],
                        yAxes: [{
                            type: '$(cur_scale)'
                        }]
                    },
                    title: {
                        display: true,
                        text: "$(cur_headers[cur_index])"
                    },
                    legend: {
                        display: false
                    }
                }
            });
        """
    end
    
    evaljs(cur_scope, JSString(cur_js))
end

return

In [ ]:
function update_plot()
    cur_headers, cur_data_sets = get_cur_data_sets()
    
    cur_scale = get_cur_scale()
    
    run_javascript(cur_headers, cur_data_sets, cur_scale)
end

return

In [ ]:
on(cur_set_observer) do cur_action
    toggle_button(cur_action, ".js-left-buttons")
    update_plot()
end

on(cur_scale_observer) do cur_action
    toggle_button(cur_action, ".js-right-buttons")
    update_plot()
end

return

In [ ]:
cur_script = """
    <script class="js-load-remove-tag">
        \$(".js-load-message").remove();
        \$("#appmode-busy").css("opacity","0");
        \$('pre:contains("WebIO: setting up")').parent(".output_subarea").remove();
        \$(".js-load-remove-tag").parent(".output_subarea").remove();
    </script>
"""

display(HTML(cur_script))

In [ ]:
main_node = Node(
    :div,
    Node(
        :div,
        Node(
            :canvas,
            attributes=Dict(:id => "js-tetris-chart-1")
        ),
        attributes=Dict(:class => "col-sm-6", :style => "height: 45vmin;")
    ),
    Node(
        :div,
        Node(
            :canvas,
            attributes=Dict(:id => "js-tetris-chart-2")
        ),
        attributes=Dict(:class => "col-sm-6", :style => "height: 45vmin;")
    ),
    attributes=Dict(:class => "row", :style => "margin-top: 3vh;")
)

cur_scope(
    Node(
        :div,
        Node(
            :div,
            cur_right_buttons...,
            attributes=Dict(:class => "js-right-buttons", :style => "float: right;")
        ),
        Node(
            :div,
            cur_left_buttons...,
            attributes=Dict(:class => "js-left-buttons")
        ),
        main_node,
        attributes=Dict(:id => "js-julia-dash")
    )
)

In [ ]:
update_plot()